In [ ]:
import numpy as np
import time
from tqdm import trange
import sys
import os
from tqdm import trange
sys.path.append('../python') #Path to load WI_Solver_Utils.py
import WI_Solver_utils
from WI_Solver_utils import InflatonModel, Background, Perturbations

In [ ]:
#Define Global parameters:

Mpl = 1 # Everything is in Plank units M_{pl}
g = 228.27  # SUSY relativistic degrees of freedom
a1 = np.pi**2/30*g

Ne_inflation=60 #Number of e-folds of after CMB horizon crossing
Ne_pert= 10 # Number of e-folds before CMB horizon crossing
Nruns=1024 #How many runs we average over to compute the perturbations

metric_perturbations=False
MPs_bool='wo' #without metric perturbations
negligible_epsH_etaH=True
epsH_etaH_bool='wo' #negligible epsH and etaH

n_cores=2 #Set the number of cores to whatever is available in your machine

#Time array where for \tau= \ln(k/aH): #
tau_ini = 6
tau_end = -1
N = 100000
# dtau is a negative quantity as expected since tau is decreasing over time
dtau = (tau_end - tau_ini) / N
taus = np.linspace(tau_ini, tau_end, N)


if not os.path.exists('WI_PowerSpectra_data'):
    os.makedirs('WI_PowerSpectra_data')

############################################################################################################
###                             EXAMPLE 1: MONOMIAL QUARTIC POTENTIAL                                    ###
############################################################################################################

potential_type='quartic'


lv = 10**(-14)  # value of lambda

Model = InflatonModel('monomial', [lv, 4], g, a1, Mpl)

ICs_Q0_ph0=np.loadtxt('../ICS_ph0-Q0s/ICS_ph0-Q0s_'+potential_type+'_Ne'+str(int(Ne_inflation))+'.txt')

Q0s=ICs_Q0_ph0[:,0]
ph0s=ICs_Q0_ph0[:,1]

#################################################################################################################
#   Fixed values of c and m, which define the temperature and inflaton field dependence in the dissipation rate #
#   according to: \Upsilon\propto T^c/\phi^m                                                                    #
#################################################################################################################

#As an example here we pick cval=3; mval=0
cval=3
mval=0

numQs=len(Q0s)
print('Case c='+str(int(cval))+' initiated')
file_name='../WI_PowerSpectra_data/PowerSpectra_'+str(MPs_bool)+'MPs_'+str(epsH_etaH_bool)+'epsH-etaH_'+potential_type+'_c'+str(cval)+'m'+str(mval)+'_n'+str(int(N))+'_Nr'+str(int(Nruns))+'_Ne'+str(int(Ne_inflation))+'.txt'
k=0
while k in range(numQs):
        
    #Initiate the Background module and compute the background evolution given the previously determined ICs:
    Bg = Background(Model, ph0s[k], Q0s[k])
    Bg_data=Bg.Bg_solver_tau(int(Ne_inflation+Ne_pert), 2*N, taus, Ne_inflation)
        
    #Compute the evolution of the perturbations equations by inputting the evolution of the background quantities:
        
    Ps=Perturbations(Model,N,Nruns,taus,dtau,cval,mval,Bg_data)
        
    '''Parallelized evaluation of the perturbation spectra: 
    hatR2info is a (Nruns/n_cores) x 2 array, where the first column is the <\mathcal{R}^2> values and
    the second column is the standard deviation of the <\mathcal{R}^2> values'''
        
    hatR2info= np.array([Ps.Pool_solver(range(n_cores),n_cores) for i in range(int(Nruns/n_cores))])
    hatR2_avg=np.mean(hatR2info[:,0])   #mean of the hatR2 values
    hatR2_std=np.sqrt(np.sum(hatR2info[:,1]**2))/len(hatR2info[:,1]) #standard deviation of the hatR2 values
    data1 = np.array([Q0s[k], hatR2_avg, hatR2_std])
    if not os.path.exists(file_name):
    # File doesn't exist, so create it with the new column
        datas = np.expand_dims(data1, axis=1)
    else:
    # File exists, so load the data and check if the new column already exists
        datas = np.loadtxt(file_name)
        if datas.ndim == 1:
            datas = datas.reshape(-1, 1)
    if not np.array_equal(datas[:, 0], data1):
        if not np.any(datas[0, :] == data1[0]):
        # New column doesn't exist, so add it as the first column
        # Only add if first element of new column not in first row
            location = np.searchsorted(-datas[0, :], -data1[0],'left')
            datas = np.insert(datas, location, data1, axis=1)
        else:
        # Substitute the new column for existing column
            idx = np.where(datas[0, :] == data1[0])[0][0]
            datas[:, idx] = data1
    # Save the data back to the file
    np.savetxt(file_name, datas)
    print('Q_0='+str(Q0s[k])+' done!')
    k+=1